In [1]:
import pandas as pd
import numpy as np # Importamos numpy para usar 'NaN' na limpeza

# --- PASSO 1: Definir colunas e carregar os dados ---
print("Carregando o arquivo de resultados (DADOS/RESULTADOS_2024.csv)...")

colunas_definitivas = [
    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT', 
    'NU_NOTA_REDACAO',
    # Informações da Escola
    'TP_DEPENDENCIA_ADM_ESC',
    'SG_UF_ESC',
    'TP_LOCALIZACAO_ESC',
    # Status da Redação para limpeza
    'TP_STATUS_REDACAO'
]

caminho_resultados = 'DADOS/RESULTADOS_2024.csv'

try:
    # Usando o parâmetro 'usecols' com a nossa lista definitiva
    df = pd.read_csv(
        caminho_resultados,
        sep=';',
        encoding='latin-1',
        usecols=colunas_definitivas
    )
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")
    print("Verifique se o nome do arquivo e o caminho estão corretos.")

# --- PASSO 2: Filtragem e Limpeza ---
print("\nIniciando filtragem e limpeza dos dados...")

# Filtrar apenas escolas de São Paulo
df_sp = df[df['SG_UF_ESC'] == 'SP'].copy()
print(f"-> {len(df_sp)} registros encontrados para escolas em São Paulo.")

# Remover participantes com notas nulas em matemática (um bom indicador de ausência)
df_sp.dropna(subset=['NU_NOTA_MT'], inplace=True)
print(f"-> {len(df_sp)} registros restantes após remover ausentes na prova de matemática.")

# Garantir que estamos analisando apenas redações válidas (status 1 = 'Sem problemas')
df_sp = df_sp[df_sp['TP_STATUS_REDACAO'] == 1]
print(f"-> {len(df_sp)} registros restantes após filtrar por redações válidas.")

# --- PASSO 3: Mapeamento e Criação de Categorias ---
print("\nCriando categorias para 'Tipo de Escola' e 'Localização'...")

# Mapear a dependência administrativa para 'Pública' ou 'Privada'
# 1, 2, 3 -> Pública | 4 -> Privada
df_sp['TIPO_ESCOLA'] = df_sp['TP_DEPENDENCIA_ADM_ESC'].apply(lambda x: 'Pública' if x in [1, 2, 3] else 'Privada')

# Mapear a localização para 'Urbana' ou 'Rural'
# 1 -> Urbana | 2 -> Rural
df_sp['LOCALIZACAO'] = df_sp['TP_LOCALIZACAO_ESC'].apply(lambda x: 'Urbana' if x == 1 else 'Rural')

# --- PASSO 4: Análise e Exibição dos Resultados ---
print("\nCalculando médias de desempenho...")

# Agrupar por TIPO_ESCOLA e calcular a média das notas
media_por_tipo = df_sp.groupby('TIPO_ESCOLA')[['NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']].mean().round(2)

print("\n--- MÉDIA DE NOTAS POR TIPO DE ESCOLA (SP) ---")
print(media_por_tipo)

# Análise Bônus: Agrupar por TIPO_ESCOLA e LOCALIZACAO
media_por_localizacao = df_sp.groupby(['TIPO_ESCOLA', 'LOCALIZACAO'])[['NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean().round(2)

print("\n\n--- ANÁLISE BÔNUS: MÉDIA POR TIPO E LOCALIZAÇÃO DA ESCOLA (SP) ---")
print(media_por_localizacao)

Carregando o arquivo de resultados (DADOS/RESULTADOS_2024.csv)...
Dados carregados com sucesso!

Iniciando filtragem e limpeza dos dados...
-> 285199 registros encontrados para escolas em São Paulo.
-> 217717 registros restantes após remover ausentes na prova de matemática.
-> 212639 registros restantes após filtrar por redações válidas.

Criando categorias para 'Tipo de Escola' e 'Localização'...

Calculando médias de desempenho...

--- MÉDIA DE NOTAS POR TIPO DE ESCOLA (SP) ---
             NU_NOTA_MT  NU_NOTA_LC  NU_NOTA_REDACAO
TIPO_ESCOLA                                         
Privada          620.84      569.05           749.34
Pública          515.42      530.43           624.42


--- ANÁLISE BÔNUS: MÉDIA POR TIPO E LOCALIZAÇÃO DA ESCOLA (SP) ---
                         NU_NOTA_MT  NU_NOTA_REDACAO
TIPO_ESCOLA LOCALIZACAO                             
Privada     Rural            586.11           727.22
            Urbana           621.00           749.45
Pública     Rural     